In [4]:
import pickle

# Open the file with the correct encoding
with open(r'C:\Users\sheik\Documents\School\GeorgiaTech\3_2024Fall\DL\KID_PPG\code\data\PPG_FieldStudy\S1\S1.pkl', 'rb') as file:
    data = pickle.load(file, encoding='latin1')  # Adjust encoding if needed

# close the file
file.close()

print('Showing the pickled data:')

cnt = 0
for item in data:
    print('The data ', cnt, ' is : ', item)
    cnt += 1

Showing the pickled data:
The data  0  is :  rpeaks
The data  1  is :  signal
The data  2  is :  label
The data  3  is :  activity
The data  4  is :  questionnaire
The data  5  is :  subject


In [7]:
data.keys()

dict_keys(['rpeaks', 'signal', 'label', 'activity', 'questionnaire', 'subject'])

In [23]:
data["signal"]

{'chest': {'ACC': array([[ 0.85640001, -0.06779999, -0.36559999],
         [ 0.8556    , -0.06900001, -0.3646    ],
         [ 0.852     , -0.07020003, -0.3642    ],
         ...,
         [ 0.88759995, -0.1038    , -0.27920002],
         [ 0.88619995, -0.10159999, -0.27819997],
         [ 0.88680005, -0.10140002, -0.27380002]]),
  'ECG': array([[0.01560974],
         [0.01812744],
         [0.01753235],
         ...,
         [0.21368408],
         [0.21556091],
         [0.21702576]]),
  'EMG': array([[-1.5],
         [-1.5],
         [-1.5],
         ...,
         [-1.5],
         [-1.5],
         [-1.5]]),
  'EDA': array([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]),
  'Temp': array([[-273.15],
         [-273.15],
         [-273.15],
         ...,
         [-273.15],
         [-273.15],
         [-273.15]], dtype=float32),
  'Resp': array([[ 4.4418335 ],
         [ 4.45098877],
         [ 4.47387695],
         ...,
         [-3.050

In [20]:
data["signal"]["wrist"]["ACC"].shape

(294784, 3)

In [27]:
data["signal"]["chest"]["ECG"].shape

(6448400, 1)

In [31]:
data["label"].shape, len(data["signal"]["wrist"]["BVP"]) / 8

((4603,), 73696.0)